<a href="https://colab.research.google.com/github/Manshi4952/Offensive-Content-Detection-using-Machine-Learning-and-NLP/blob/main/Offensive_Cotent_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
#

In [2]:
import re
import nltk
nltk.download('stopwords')
from nltk.util import pr
stemmer = nltk.SnowballStemmer("english")
from nltk.corpus import stopwords
import string
stopword=set(stopwords.words("english"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [7]:
from google.colab import files
uploaded = files.upload()

Saving english_dataset.tsv to english_dataset.tsv


In [8]:
df = pd.read_csv("english_dataset.tsv", sep='\t')
print(df.head())

      text_id                                               text task_1  \
0  hasoc_en_1  #DhoniKeepsTheGlove | WATCH: Sports Minister K...    NOT   
1  hasoc_en_2  @politico No. We should remember very clearly ...    HOF   
2  hasoc_en_3  @cricketworldcup Guess who would be the winner...    NOT   
3  hasoc_en_4  Corbyn is too politically intellectual for #Bo...    NOT   
4  hasoc_en_5  All the best to #TeamIndia for another swimmin...    NOT   

  task_2 task_3  
0   NONE   NONE  
1   HATE    TIN  
2   NONE   NONE  
3   NONE   NONE  
4   NONE   NONE  


In [9]:
df['labels'] = df['task_1'].map({"HOF":"Hate Speech Detected", "HATE":"Hate Speech Detected", "OFFN":"Offensive language detected",
                                 "PRFN":"Offensive language detected", "NOT":"No hate and offensive speech",
                                 "NONE":"No hate and offensive speech"})
print(df.head())

      text_id                                               text task_1  \
0  hasoc_en_1  #DhoniKeepsTheGlove | WATCH: Sports Minister K...    NOT   
1  hasoc_en_2  @politico No. We should remember very clearly ...    HOF   
2  hasoc_en_3  @cricketworldcup Guess who would be the winner...    NOT   
3  hasoc_en_4  Corbyn is too politically intellectual for #Bo...    NOT   
4  hasoc_en_5  All the best to #TeamIndia for another swimmin...    NOT   

  task_2 task_3                        labels  
0   NONE   NONE  No hate and offensive speech  
1   HATE    TIN          Hate Speech Detected  
2   NONE   NONE  No hate and offensive speech  
3   NONE   NONE  No hate and offensive speech  
4   NONE   NONE  No hate and offensive speech  


In [10]:
df = df[['text','labels']]
print(df.head())

                                                text  \
0  #DhoniKeepsTheGlove | WATCH: Sports Minister K...   
1  @politico No. We should remember very clearly ...   
2  @cricketworldcup Guess who would be the winner...   
3  Corbyn is too politically intellectual for #Bo...   
4  All the best to #TeamIndia for another swimmin...   

                         labels  
0  No hate and offensive speech  
1          Hate Speech Detected  
2  No hate and offensive speech  
3  No hate and offensive speech  
4  No hate and offensive speech  


In [11]:
def clean(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
df["text"] = df["text"].apply(clean)
print(df.head())



                                                text  \
0  dhonikeepstheglov  watch sport minist kiren ri...   
1  politico rememb clear  admit treason  trumpisa...   
2  cricketworldcup guess would winner      team g...   
3  corbyn polit intellectu borisjohnsonshouldnotb...   
4  best teamindia anoth swim competit sunday paki...   

                         labels  
0  No hate and offensive speech  
1          Hate Speech Detected  
2  No hate and offensive speech  
3  No hate and offensive speech  
4  No hate and offensive speech  


In [12]:
x = np.array(df["text"])
y = np.array(df["labels"])
cv = CountVectorizer()
X = cv.fit_transform(x)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf = DecisionTreeClassifier()
clf.fit(X_train,y_train)





DecisionTreeClassifier()

In [14]:
test_data = "you are bastard"
df = cv.transform([test_data]).toarray()
output = clf.predict(df)
print(output)

['Hate Speech Detected']


In [15]:
test_data = "Have a nice day"
df = cv.transform([test_data]).toarray()
output = clf.predict(df)
print(output)

['No hate and offensive speech']


In [26]:
test_data = "you are an idiot"
df = cv.transform([test_data]).toarray()
output = clf.predict(df)
print(output)

['Hate Speech Detected']
